In [2]:
import os
import numpy as np
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import transforms
from torchmetrics import (
    Accuracy,
    MetricCollection,
    JaccardIndex,
    F1Score,
    MeanSquaredError,
)
import segmentation_models_pytorch as smp
from pytorch_toolbelt.losses import JaccardLoss, BinaryFocalLoss
from torch.utils.data import Dataset, random_split, DataLoader

from pytorch_lightning import Trainer
from typing import Any, Tuple, Optional, Callable, cast

import matplotlib.pyplot as plt
import wandb
import tqdm as notebook_tqdm
import random

/Users/sofija/miniforge3/envs/torch_m1/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

In [4]:
import segmentation_models_pytorch as smp

In [5]:
pl.seed_everything(27)

Global seed set to 27


27

In [6]:
wandb.login()

wandb: Currently logged in as: sofstef (trees). Use `wandb login --relogin` to force relogin


True

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
%load_ext lab_black

In [9]:
os.chdir("/Users/sofija/Ai4er/mres/tree-segmentation")
os.getcwd()

'/Users/sofija/Ai4er/mres/tree-segmentation'

In [10]:
from src.datasets import TreeSegments
from src.evaluation import BinaryIoU
from src.datamodules import TreeDataModule
from src.models import UNet, SegModel
from src.evaluation import LogPredictionSamplesCallback

In [11]:
dm = TreeDataModule(
    data_dir="data/raw/samples/",
    target_dir="data/segments/",
    test_data_dir="data/test_samples",
    test_target_dir="data/test_segments/",
    batch_size=8,
    num_workers=0,
    drop_last_batch=True,
)

In [12]:
best_path = "lightning_logs/colab_ckpt/epoch=78-step=632-v1.ckpt"

In [13]:
model = SegModel.load_from_checkpoint(checkpoint_path=best_path)

In [14]:
trainer = Trainer()

/Users/sofija/miniforge3/envs/torch_m1/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [21]:
# dropout, seeing whether frozen/unfrozen weights make a difference if this is possible here
sweep_config = {
    "entity": "trees",
    "project": "Trees",
    "method": "random",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "encoder_name": {"values": ["resnet18", "resnet34", "resnet50"]},
        # dropout is not directly specified in the smp models but through aux parameters
        # "dropout": {"values": [0.3, 0.4, 0.5]},
        "batch_size": {
            "distribution": "q_log_uniform_values",
            "q": 8,
            "min": 8,
            "max": 64,
        },
        "lr": {
            # a flat distribution between 0.0001 and 0.01
            "distribution": "uniform",
            "min": 1e-4,
            "max": 1e-1,
            # log uniform distribution between exp(min) and exp(max)
            # "distribution": "log_uniform",
            # "min": -9.21,  # exp(-9.21) = 1e-4
            # "max": -4.61,  # exp(-4.61) = 1e-2
        },
        "epochs": {"value": 5},
        "encoder_weights": {"value": "imagenet"},
        "loss": {"value": "dice"},
        "num_classes": {"value": 1},
        "in_channels": {"value": 1},
        "jaccard_average": {"value": "macro"},
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 3,
    },
}

In [22]:
sweep_id = wandb.sweep(sweep_config, project="Trees")

Create sweep with ID: kja7hfs1
Sweep URL: https://wandb.ai/trees/Trees/sweeps/kja7hfs1


Questions about setting sweeps up:

1. Which search strategy should I use? 
2. Should batch size be passed explicitly to datamodule (example doesn't seem to do that)?
3. Should I vary the number of epochs?

In [23]:
def sweep_iteration():
    # set up W&B logger
    wandb.init()  # required to have access to `wandb.config`
    wandb_logger = WandbLogger()

    # setup data
    dm = TreeDataModule(
        data_dir="data/train/samples/",
        target_dir="data/train/segments/",
        test_data_dir="data/test/samples",
        test_target_dir="data/test/segments/",
        batch_size=wandb.config.batch_size,
        drop_last_batch=True,
    )

    # setup model - note how we refer to sweep parameters with wandb.config
    model = SegModel(
        in_channels=wandb.config.in_channels,
        encoder_name=wandb.config.encoder_name,
        encoder_weights=wandb.config.encoder_weights,
        num_classes=wandb.config.num_classes,
        loss=wandb.config.loss,
        ignore_zeros=None,
        lr=wandb.config.lr,
        jaccard_average=wandb.config.jaccard_average,
    )

    # setup Trainer
    trainer = Trainer(
        logger=wandb_logger,
        # callbacks=callbacks,
        max_epochs=wandb.config.epochs,
    )

    # train
    trainer.fit(model, dm)

In [24]:
wandb.agent(sweep_id, function=sweep_iteration, count=5)

wandb: Agent Starting Run: o6xd5bay with config:
wandb: 	batch_size: 16
wandb: 	encoder_name: resnet18
wandb: 	encoder_weights: imagenet
wandb: 	epochs: 100
wandb: 	in_channels: 1
wandb: 	jaccard_average: macro
wandb: 	loss: dice
wandb: 	lr: 0.00981975863692553
wandb: 	num_classes: 1


/Users/sofija/miniforge3/envs/torch_m1/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name       | Type             | Params
-------------------------------------------------
0  | net        | Unet             | 14.3 M
1  | loss       | DiceLoss         | 0     
2  | train_acc  | Accuracy         | 0     
3  | val_acc    | Accuracy         | 0     
4  | test_acc   | Accuracy         | 0     
5  | train_jacc | JaccardIndex     | 0     
6  | val_jacc   | JaccardIndex     | 0     
7  | test_jacc  | JaccardIndex     | 0     
8  | train_f1   | F1Score          | 0     
9  | val_f1     | F1Score          

/Users/sofija/miniforge3/envs/torch_m1/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/Users/sofija/miniforge3/envs/torch_m1/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/sofija/miniforge3/envs/torch_m1/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1927: PossibleUserWarning: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:  80%|████████████████████████▊      | 4/5 [00:07<00:01,  1.98s/it, loss=0.732, v_num=5bay]
Validation: 0it [00:00, ?it/s]
Epoch 1:  80%|████████████████████████▊      | 4/5 [00:07<00:01,  1.79s/it, loss=0.628, v_num=5bay]
Validation: 0it [00:00, ?it/s]
Epoch 2:  80%|████████████████████████▊      | 4/5 [00:07<00:01,  1.79s/it, loss=0.573, v_num=5bay]
Validation: 0it [00:00, ?it/s]
Epoch 3:  80%|████████████████████████▊      | 4/5 [00:07<00:01,  1.78s/it, loss=0.516, v_num=5bay]
Validation: 0it [00:00, ?it/s]
Epoch 4:  80%|████████████████████████▊      | 4/5 [00:07<00:01,  1.75s/it, loss=0.465, v_num=5bay]
Validation: 0it [00:00, ?it/s]
Epoch 5:  80%|████████████████████████▊      | 4/5 [00:08<00:02,  2.16s/it, loss=0.368, v_num=5bay]
Validation: 0it [00:00, ?it/s]
Epoch 6:  80%|████████████████████████▊      | 4/5 [00:07<00:01,  1.93s/it, loss=0.312, v_num=5bay]
Validation: 0it [00:00, ?it/s]
Epoch 7:  80%|████████████████████████▊      | 4/5 [00:07<00:01,  1.88s/it, loss=0.

...

Run tjbo9ubu errored: ZeroDivisionError('float division by zero')
wandb: ERROR Run tjbo9ubu errored: ZeroDivisionError('float division by zero')
wandb: Agent Starting Run: qg4eqlhg with config:
wandb: 	batch_size: 48
wandb: 	encoder_name: resnet50
wandb: 	encoder_weights: imagenet
wandb: 	epochs: 10
wandb: 	in_channels: 1
wandb: 	jaccard_average: macro
wandb: 	loss: dice
wandb: 	lr: 0.00951808682450227
wandb: 	num_classes: 1


/Users/sofija/miniforge3/envs/torch_m1/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name       | Type             | Params
-------------------------------------------------
0  | net        | Unet             | 32.5 M
1  | loss       | DiceLoss         | 0     
2  | train_acc  | Accuracy         | 0     
3  | val_acc    | Accuracy         | 0     
4  | test_acc   | Accuracy         | 0     
5  | train_jacc | JaccardIndex     | 0     
6  | val_jacc   | JaccardIndex     | 0     
7  | test_jacc  | JaccardIndex     | 0     
8  | train_f1   | F1Score          | 0     
9  | val_f1     | F1Score          

Sanity Checking: 0it [00:00, ?it/s]

/Users/sofija/miniforge3/envs/torch_m1/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:127: UserWarning: Total length of `DataLoader` across ranks is zero. Please make sure this was your intention.
  rank_zero_warn(
/Users/sofija/miniforge3/envs/torch_m1/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Run qg4eqlhg errored: ZeroDivisionError('float division by zero')
wandb: ERROR Run qg4eqlhg errored: ZeroDivisionError('float division by zero')


In [25]:
wandb.finish()